# 2023年度 深層学習演習 最終課題
## 23vr008n 高林 秀

## 問題設定
### 選択した課題
* (a) : テーブルデータ、画像データ、系列データのいずれか(※ 広い意味で捉えて良い) を用意し、回帰もしくは分類タスクを設定して深層学習モデルにより学習、評価を行え。よりよい結果が得られるよう試行錯誤し、過程も含めて示すこと。 
### 設定したタスク
* ポケモンの画像分類-オーキドモデルを作る-
 * 下記データセットを使用する。
 * 149クラスある
* https://www.kaggle.com/datasets/echometerhhwl/pokemon-gen-1-38914

## データセットについて
学習で使うデータセットはkaggleからダウンロードしたものを使用する。

このデータは、**149種類のポケモンの画像**が入っている。アニメ画像の切り抜きや、手書きで描いたポケモンの画像などが含まれている。

各ポケモン毎にフォルダが分かれており、その中に画像が入っている。
画像の層数は全部で**35627枚**である。

## 使用するモデルについて
本課題では事前学習済みのモデルをファインチューニングすることで、モデルを作成する。

今回使用するモデルは以下の通りである。
* モデル名：EfficientNetV2
* TODO: モデルの説明を書く

## 評価方法
ダウンロードしたデータセットを、学習用とテストセットに分け、テストセットを用いて評価を行う。

なお、評価方法は、**Accuracy**を用いる。

## 実行環境
本稿で作成したモデルは、ローカルマシン上で構築した。以下に実行環境を示す。

* OS: Windows 11
* CPU: Intel(R) Core(TM) i7-13700H @ 2.40GHz
* GPU: NVIDIA GeForce RTX 4060 Laptop GPU @ 8GB
* RAM: 32GB
* 開発環境
    * Python 3.9.6
    * PyTorch 1.9.1 + cu111
    * Torchvision 0.10.1 + cu111

## サンプルデータの表示
学習データで使用する画像は次のようなもので、単純にポケモンのみが写っているものもあれば、背景に人間が写っているものや、手書きのもの、カード状のものまで様々である。

<img src="archive/data/Raichu/0c7e0a91bf65facbbc2d2c06b55f1bf2.jpg" style="width: 250px;"/>
<img src="archive/data/Raichu/ee730b7e1e47a1aafd94476016875344.jpg" style="width: 250px;"/>
<img src="archive/data/Vileplume/0ac8fd551dd66cda9f03e6124363b071.png" style="width: 250px;"/>
<img src="archive/data/Zapdos/0b3cf9fd603a437e50e76222ce0db245.jpg" style="width: 250px;"/>


## 実装/モデル構築

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
from torch.utils.data import random_split
import torch.nn as nn
import torchvision.models as models


In [ ]:
#CUDAが有効化どうかの確認
print(f"CUDA is Available : {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
DATA_DIR = "archive/data"

# 画像の前処理（リサイズ、テンソル変換など）
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

#各画像が格納されたサブディレクトリの名前をクラス名として扱う。
all_dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform) #datasets.ImageFolderは、自動的にサブディレクトリをクラス名として扱う。

TOTAL_SIZE = len(all_dataset)
print(f"TOTAL_SIZE = {TOTAL_SIZE}")
USE_SIZE = TOTAL_SIZE
print(f"USE_SIZE = {USE_SIZE}")


dataset, _ = random_split(all_dataset, [USE_SIZE, TOTAL_SIZE - USE_SIZE])

TRAIN_RATO = 0.7
VAL_RATIO = 0.2

train_size = int(USE_SIZE * TRAIN_RATO)
val_size = int(USE_SIZE * VAL_RATIO)
test_size = int(USE_SIZE - train_size - val_size)
split_size = [train_size, val_size, test_size]
print(f"split_size ={split_size}")
print(f"sum = {sum(split_size)}")
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Class = {all_dataset.classes}")
print(f"Class Size = {len(all_dataset.classes)}")

CLASS_SIZE = len(all_dataset.classes)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(30, 30))
for i, (images, labels) in enumerate(train_loader):
    if i < 9:  # 最初の9枚の画像のみ表示
        img = images[0]
        label = labels[0]
        ax = axs[i // 3, i % 3]
        ax.imshow(img.permute(1, 2, 0))  # チャンネル次元の順序を変更
        ax.set_title(f'Label: {label}')
        ax.axis('off')
    else:
        break

plt.show()

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #NOTE:調べたところtorchで読み込むとPILの画像破損判定に引っかかる。強制的に読み込む。
def show_distribution(loader, title="") -> list[int]:
    count = [0] * CLASS_SIZE
    for _, label in tqdm(loader, desc="Generating Distribution..."):
        for l in label:
            count[l] += 1
    plt.bar(range(CLASS_SIZE), count)
    plt.xticks(rotation=30)
    plt.title(title)
    plt.show()
    return count

train_dist = show_distribution(train_loader, "TrainData Distribution")
test_dist = show_distribution(test_loader, "TestData Distribution")

In [ ]:
#ダウンサンプリング

class BalancedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.indices = self.balance_classes()

    def balance_classes(self):
        # 各クラスのインデックスを格納する
        class_indices = defaultdict(list)
        for idx, (_, label) in tqdm(enumerate(self.dataset), desc="Collectioning Index..."):
            class_indices[label].append(idx)

        min_size = min(len(indices) for indices in class_indices.values())

        # 各クラスからランダムにmin_size個のインデックスを選択
        balanced_indices = []
        for indices in tqdm(class_indices.values(), desc="Down Sampling..."):
            balanced_indices.extend(np.random.choice(indices, min_size, replace=False))

        return balanced_indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)
    
d_sampled_dataset = BalancedDataset(all_dataset)
print(f"down sampled_dataset size = {len(d_sampled_dataset)}")


In [ ]:
#ローダーの再定義

train_size = int(len(d_sampled_dataset) * TRAIN_RATO)
val_size = int(len(d_sampled_dataset) * VAL_RATIO)
test_size = int(len(d_sampled_dataset) - train_size - val_size)
split_size = [train_size, val_size, test_size]
print(f"split_size ={split_size}")

d_sampled_train_dataset, d_sampled_val_dataset, d_sampled_test_dataset = random_split(d_sampled_dataset, split_size)

d_sampled_train_loader = DataLoader(d_sampled_train_dataset, batch_size=32, shuffle=True)
d_sampled_val_loader = DataLoader(d_sampled_val_dataset, batch_size=32, shuffle=False)
d_sampled_test_loader = DataLoader(d_sampled_test_dataset, batch_size=32, shuffle=False)

d_sampled_train_dist = show_distribution(d_sampled_train_loader, "Down Sampled TrainData Distribution")
d_sampled_test_dist = show_distribution(d_sampled_test_loader, "Down Sampled TestData Distribution")


In [ ]:
#モデルのロード

model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(in_features=4096, out_features=CLASS_SIZE, bias=True)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def calculate_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
EPOCH = 3

for epoch in range(EPOCH):
    model.train()
    # 訓練フェーズ
    for idx, [inputs, labels] in enumerate(d_sampled_test_loader):  # train_loaderはトレーニングデータのローダー
        inputs, labels = inputs.to(device), labels.to(device)
        print("Training iteration: ", idx+1, end="\r")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    accuracy = calculate_accuracy(model, d_sampled_val_loader)
    print(f"Epoch {epoch}: accuracy = {accuracy}")

In [ ]:
#モデルの保存
def save_model(model, path):
    torch.save(model.state_dict(), path)

save_model(model, "./Models/20240129.pth")